In [46]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":500,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
    "stop_sequences":["[/INST]"],
}

In [69]:
proxy = "proxy.us.ibm.com:8080"

In [103]:
access_token = ""

def get_key():
    return access_token

def connect():
    # Prepare the payload and headers
    payload = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key
    }
    headers = {
        'Content-Type': "application/x-www-form-urlencoded"
    }

    # Make a POST request while ignoring SSL certificate verification
    try:
        response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
        
        # Check if the request was successful
        response.raise_for_status()

        # Parse the JSON response
        decoded_json = response.json()
        access_token = decoded_json["access_token"]
        return access_token
        # print(f"Access Token: {access_token}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

access_token = connect()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [48]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_13B_CHAT_V2 = "ibm/granite-13b-chat-v2"
GRANITE_13B_INSTRUCT_V2 = "ibm/granite-13b-instruct-v2"
GRANITE_20B_MULTILINGUAL = "ibm/granite-20b-multilingual"
LLAMA_3_70B_INSTRUCT = "meta-llama/llama-3-70b-instruct"

def watsonx_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {get_key()}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


some basic classes

In [21]:
class Agent:
    name = "Agent"
    model = "ibm/granite-13b-chat-v2"
    instructions = ["You are a helpful agent."]
    functions = []
    tool_choice = None
    parallel_tool_calls = True


class Response:
    messages = []
    agent = None
    context_variables = {}


class Result:
    value = ""
    agent = None
    context_variables = {}

In [101]:
def gen_plan(requirement):
    prompt = f"""[INST]
please generate a concise and solid plan to satisfy the requirement provided.
-understand the requirement in detail.
-list out what knowlege you need to satisfy the requirement.
-think logically.
-think step by step.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-leverage tool if possible.
-dont guess.
-please breakdown complex task to simple tasks.
-DONT generate result of the steps.
-**DONT** generate **Final Result:**
<<SYS>>
requirement: `{requirement}`
<</SYS>>
[/INST]plan:"""

    return watsonx_gen(prompt,LLAMA_3_70B_INSTRUCT).replace("```","")

In [117]:
def gen_summary(requirement,answer):
    prompt = f"""[INST]
please generate a complete sentence summary base on the answer provided only.
-use simple english.
-dont guess.
<<SYS>>
requirement: `{requirement}`
answer: `{answer}`
<</SYS>>
[/INST]answer summary:"""

    return watsonx_gen(prompt,LLAMA_3_70B_INSTRUCT).replace("```","")

In [81]:
tools = """[
    get_hko_weather()->str,
    query_duckduckgo(query:str)->str,
    search_arxiv(query:str)->str,
    query_wikipedia(title:str)->str
]"""

In [94]:
def gen_code(requirement,plan):
    prompt = f"""[INST]
please generate python code that execute the plan to fulfill the requirement.
-ensure the python generated is executable.
-clean up if any unneccessary characters.
-you can use the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem.
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-generate the code directly.
-generate the code only.
<<SYS>>
requirement: `{requirement}`
plan: `{plan}`
tools: {tools}
<</SYS>>
[/INST]
```python
"""
    
    #-dont define function.

    return watsonx_gen(prompt,LLAMA_3_70B_INSTRUCT).replace("```","")
    # print(code)

In [123]:
def revise_code(code, plan, error):
    prompt = f"""[INST]
revise the code provided.
-review the problem in the code and the error.
-prevent the error from the code.
-you can use the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem. 
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-generate the code directly.
-generate the code only.
<<SYS>>
code: `{code}`
plan: `{plan}`
error: `{error}`
tools: {tools}
<</SYS>>
[/INST]
fixed code:```python"""

    answer = watsonx_gen(prompt,LLAMA_3_70B_INSTRUCT).replace("```","")
    return answer

In [57]:
import io
import sys
import traceback
from IPython.core.interactiveshell import InteractiveShell
def call_tool(module_import, function_call, max_traceback_lines=5):
    # Create StringIO objects to capture standard output and error output
    output_stream = io.StringIO()
    error_stream = io.StringIO()

    # Get the current interactive shell
    shell = InteractiveShell.instance()

    # Redirect stdout and stderr
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    sys.stdout = output_stream
    sys.stderr = error_stream

    output = None
    error = None

    # Using exec to import the module and evaluate the function call
    try:
        local_namespace = {}
        exec(module_import, local_namespace, local_namespace)  # Dynamically import the module
        print(f"|{function_call}|")
        exec(function_call, local_namespace, local_namespace)  # Evaluate the function call
    except ImportError as e:
        error = f"Import Error: {str(e)}"
    except Exception as e:
        # Capture the traceback for more detailed error information
        full_traceback = traceback.format_exc().strip()
        # Limit the traceback to the last 'max_traceback_lines' lines
        limited_traceback = "\n".join(full_traceback.splitlines()[-max_traceback_lines:])
        error = f"Unexpected Error: {str(e)}\n{limited_traceback}"
    finally:
        # Reset stdout to its original state
        sys.stdout = old_stdout
        sys.stderr = old_stderr

    # Get the output from the output_stream
    output = output_stream.getvalue().strip()
    error_output = error_stream.getvalue().strip()

    # Check the output or error
    if error:
        return f"Output: {output}", f"Error: {error}"
    elif error_output:
        return f"Output: {output}", f"Error Output: {error_output}"
    else:
        return f"Output: {output}", None

In [58]:
def eval_output(question, stdout):
    prompt = f"""[INST]classify if stdout include error.
-show sucess or fail only, but nothing else.
-show success if include no error. and end the generation.
-show fail if include error. and end the generation.
<<SYS>>
question: `{question}`
stdout: `{stdout}`
<</SYS>>
[/INST]success or fail:"""
    success = watsonx_gen(prompt,LLAMA_3_70B_INSTRUCT)
    # print(success)
    return success

In [59]:
def agentic_prompt(requirement, max_turns=10,debug=False):
    # Generate initial action and code
    plan = gen_plan(requirement)
    if debug:
        print(f"#1: {plan}")
    code = gen_code(requirement, plan)
    if debug:
        print(f"#2: <|{code}|>")

    output = ""

    # Loop until the review output is empty
    for turn in range(max_turns):
        print(f"turn #{turn+1} ---")
        # output = exec_with_output(code)
        output, error = call_tool("",code)
        if debug:
            print(f"#3: {output}")
        
        if error is None:
            break
        
        if debug:
            print(f"error {error}")
        # review = eval_output(requirement, output)
        # if debug:
        #     print(f"#4: {review}")
        
        # if "success" in review:  # Exit if review output is empty
        #     # print(f"#final result: {output}")
        #     break
        
        revised = revise_code(code, plan, error)
        if debug:
            print(f"#5: {revised}")

        # Update code for the next iteration
        code = revised

    summary = gen_summary(requirement,output)
    return summary

In [124]:
output = agentic_prompt("how is weather today in Hong Kong, base on HKO",debug=True)
print(output)

#1:  

1. **Understand the requirement**: 
   - The requirement is to find out the current weather in Hong Kong based on the Hong Kong Observatory (HKO).

2. **List out what knowledge is needed**: 
   - Knowledge of the current weather in Hong Kong.
   - Knowledge of the Hong Kong Observatory (HKO) and its weather reporting system.

3. **Think logically and step-by-step**: 
   - Step 1: Identify a reliable source of weather information for Hong Kong.
   - Step 2: Check if the source is affiliated with the Hong Kong Observatory (HKO).
   - Step 3: Access the source and retrieve the current weather information for Hong Kong.
   - Step 4: Verify the accuracy of the information.

4. **Leverage tools if possible**: 
   - Use a search engine (e.g., Google) to find the official website of the Hong Kong Observatory (HKO).
   - Use the website to retrieve the current weather information for Hong Kong.

5. **Break down complex tasks into simple tasks**: 
   - Task 1: Find the official website of

In [142]:
output = agentic_prompt("what is the population in hong kong",debug=True)
print(output)

#1:  

1. Identify the reliable sources of information for Hong Kong's population data.
   - Knowledge needed: List of reliable sources (e.g. government websites,,erable international organizations)

2. Determine the most recent and accurate population data available.
   - Knowledge needed: Understanding of data collection methods and frequency of updates

3. Utilize online tools or databases to access the population data.
   - Knowledge needed: Familiarity with online tools and databases (e.g. World Bank, Wikipedia, Hong Kong Government website)

4. Verify the data by cross-checking with multiple sources.
   - Knowledge needed: Understanding of data validation techniques and critical thinking

5. Provide the most recent and accurate population data for Hong Kong.
   - Knowledge needed: Ability to interpret and present data accurately
#2: <|def get_population_hong_kong():
    query = "Hong Kong population"
    data = query_wikipedia(query)
    start_idx = data.find("As of") + 6
    end

In [143]:
output = agentic_prompt("how many prime number with in 10000")
print(output)

turn #1 ---
 There are 1229 prime numbers within 10000.


In [144]:
output = agentic_prompt("how many R in straberry, not regarding to upper lower case")
print(output)

turn #1 ---
 There are 3 R's in the word "strawberry".


In [145]:
output = agentic_prompt("in math, compare 9.11 with 9.8")
print(output)

turn #1 ---
 9.11 is not greater than 9.8.


In [146]:
output = agentic_prompt("tell me Hong Kong time")
print(output)

turn #1 ---
 The current time in Hong Kong is 2024-10-19 14:11:23 HKT+0800.


In [ ]:
active_agent = None
context_variables = []
history = [] 
init_len = 0

max_turns = 10
model_id = GRANITE_20B_CODE_INSTRUCT
tools = []
messages = []
active_agent = Agent(model_id,"do math")
while len(history) - init_len < max_turns:
    message = ""
    partial_response = watsonx_gen(message,history)
    history.append(message)
    history.append(partial_response.message)
    context_variables.update(partial_response.ontext_variable)
    active_agent = partial_response.agent

In [ ]:
def clock():
    print("12pm")
    return "12pm"

def calculate(expression):
    return eval(expression)

prompt = f"""[INST]
tell me 3434 * 21 / 32
-for any math question, dont base on memory. ride on tool.
-tell what you confidence to answer, dont guess.
-generate the python code to use tool in need.
-only use the tool provided.
-dont repeat.
-generate the answer or code only.
<<SYS>>
tools: [
    clock()->str #get time,
    abc(code:str)->str #get stock price, 
    calculator(expression:str)->str,
    browser(url:str)->str
    ]
<</SYS>>
[/INST]answer:
"""

answer = watsonx_gen(prompt,LLAMA_3_70B_INSTRUCT)
print(answer)
out = call_tool("",answer.strip())
print(out)

In [ ]:
print(3434 * 21 / 32)

In [ ]:
def clock():
    print("12pm")
    return "12pm"

def ask_granite(question):
    prompt = f"""<|system|>
-tell what you confidence to answer only, dont guess.
-select the right tool.
-dont base on memory. call the right tool in contenxt.
[TOOLS]
tools: [
    clock()->str #get time,
    abc(code:str)->str #get stock price, 
    calculator(expression:str)->str # do math,
    browser(url:str)->str #find informatino
    ]
[/TOOLS]
-generate the python code to use tool in need.
-only use the tool provided.
-dont repeat.
-generate the answer or code only.
-dont generate result.
-dont generate Calculation steps.
-generate the code only.
<|user|>
{question}
<|assistant|>
```python"""

    answer = watsonx_gen(prompt,GRANITE_13B_CHAT_V2)
    return answer

answer = ask_granite("tell me the content of cnn.com today")
print(answer)

answer = ask_granite("tell me 320*232 / 2322")
print(answer)

answer = ask_granite("tell me about paris")
print(answer)

In [ ]:
#try on a task
#give some tools
#run the tool
#get the result
#check if the answer satisfy the task
#do until limit
#put all in context
#put all in history

In [ ]:
def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Haikus.",
)

response = run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])

In [ ]:
messages = []
agent = starting_agent

while True:
    user_input = input("\033[90mUser\033[0m: ")
    messages.append({"role": "user", "content": user_input})

    response = run(
        agent,
        messages,
        context_variables or {}
    )

    response = process_and_print_streaming_response(response)

    messages.extend(response.messages)
    agent = response.agent